### 토큰화

In [11]:
# !pip install nltk 처음한번

import nltk
from nltk.collocations import *
from nltk.tokenize import RegexpTokenizer
import pandas as pd

In [12]:
df = pd.read_csv('preprocessedFiles/preprocessed_done.csv')

#잘 생성됐는지 확인
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1574 entries, 0 to 1573
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Country Code     1574 non-null   object
 1   School Name      1574 non-null   object
 2   Review Link      1574 non-null   object
 3   Review Title     1574 non-null   object
 4   Review Content   1574 non-null   object
 5   cleaned_content  1574 non-null   object
 6   preprocessed     1574 non-null   object
dtypes: object(7)
memory usage: 86.2+ KB


,Country Code,School Name,Review Link,Review Title,Review Content,cleaned_content,preprocessed
0,AT,FH Kufstein Tirol University of Applied Sciences,https://oia.yonsei.ac.kr/partner/expReport.asp...,"소중한 경험, 잊지 못할 기억, 쿠프슈타인",다만 집주인이 관리를 잘 안 해주고 규칙도 명확하지 않아서 청소나 화장실 이용 등...,다만 집주인이 관리를 잘 안 해주고 규칙도 명확하지 않아서 청소나 화장실 이용 등...,"[['집주인', '관리', '안', '규칙', '청소', '화장실', '이용', '..."
1,AT,FH Kufstein Tirol University of Applied Sciences,https://oia.yonsei.ac.kr/partner/expReport.asp...,언제나 마음속의 고향인 쿠프슈타인,"비슷한 가격대에 비교적 더 자유롭게 외부에서 자취도 할 수 있지만, 기숙사만의 좋...",비슷한 가격대에 비교적 더 자유롭게 외부에서 자취도 할 수 있지만 기숙사만의 좋...,"[['가격', '비교', '더', '외부', '자취', '기숙사', '점', '사전..."
2,AT,FH Kufstein Tirol University of Applied Sciences,https://oia.yonsei.ac.kr/partner/expReport.asp...,쿠프슈타인에서의 한 학기,"사람 사는곳 다 똑같고, 생각한대로 다 되지 않는다는 이야기는 익히 듣고 와서 그...",사람 사는곳 다 똑같고 생각한대로 다 되지 않는다는 이야기는 익히 듣고 와서 그...,"[['사람', '곳', '생각', '대로', '이야기', '그게', '크게', '스..."
3,AT,FH Kufstein Tirol University of Applied Sciences,https://oia.yonsei.ac.kr/partner/expReport.asp...,여유롭고 소중했던 쿠프슈타인에서의 시간,불안해서 국제처에도 계속 문의하고 쿠프슈타인 대학교에도 계속 문의했었는데 계속 기...,불안해서 국제처에도 계속 문의하고 쿠프슈타인 대학교에도 계속 문의했었는데 계속 기...,"[['국제', '처', '계속', '문의', '쿠프슈타인', '대학교', '계속',..."
4,AT,Johannes Kepler Universitat Linz,https://oia.yonsei.ac.kr/partner/expReport.asp...,"유럽의 심장, 평화롭고 선명했던 린츠를 그리워하며",흐린 날씨가 지속되면 처음에는 조금 우울하지만 그래도 이것이 유럽의 겨울이구나 하...,흐린 날씨가 지속되면 처음에는 조금 우울하지만 그래도 이것이 유럽의 겨울이구나 하...,"[['날씨', '지속', '처음', '유럽', '겨울'], ['오스트리아', '문화..."


In [13]:
# Assuming df['cleaned_content'] contains the preprocessed reviews
tokenizer = RegexpTokenizer(r'\w+')

# Tokenize each review and convert to lowercase
df['tokens'] = df['preprocessed'].apply(lambda x: tokenizer.tokenize(x.lower()))

df['tokens']

0                 [집주인, 관리, 안, 규칙, 청소, 화장실, 이용, 문제, 스트레스]
1       [가격, 비교, 더, 외부, 자취, 기숙사, 점, 사전, 살, 곳, 실제, 보지, ...
2                   [사람, 곳, 생각, 대로, 이야기, 그게, 크게, 스트레스, 요]
3       [국제, 처, 계속, 문의, 쿠프슈타인, 대학교, 계속, 문의, 계속, 연락, 나중...
4       [날씨, 지속, 처음, 유럽, 겨울, 오스트리아, 문화, 충격, 충격, 오스트리아,...
                              ...                        
1569              [때문, 영어, 렁증, 발표, 공포증, 가지, 전형, 처음, 스트레스]
1570                                             [템포, 박자]
1571                   [사람, 관계, 지속, 때문, 적극, 성격, 스트레스, 수도]
1572    [달, 정도, 스트레스, 좌절, 어회, 책, 전자사전, 항상, 붙이, 점점, 적응,...
1573    [처음, 언어, 뿐, 환경, 변화, 스트레스, 정말, 처음, 시작, 수영, 수업, ...
Name: tokens, Length: 1574, dtype: object

### 토픽 모델링

In [14]:
!pip install gensim

from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# 사전 생성 및 코퍼스 준비
# 사전 생성
dictionary = corpora.Dictionary(df['tokens'])

# 코퍼스 생성
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

2024-05-29 23:29:47,256 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-05-29 23:29:47,283 : INFO : built Dictionary<3269 unique tokens: ['관리', '규칙', '문제', '스트레스', '안']...> from 1574 documents (total 20160 corpus positions)
2024-05-29 23:29:47,285 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<3269 unique tokens: ['관리', '규칙', '문제', '스트레스', '안']...> from 1574 documents (total 20160 corpus positions)", 'datetime': '2024-05-29T23:29:47.285157', 'gensim': '4.3.2', 'python': '3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'created'}


In [22]:
num_topics = 6
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2024-05-30 00:17:00,952 : INFO : using autotuned alpha, starting with [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667]
2024-05-30 00:17:00,957 : INFO : using serial LDA version on this node


2024-05-30 00:17:00,971 : INFO : running online (multi-pass) LDA training, 6 topics, 20 passes over the supplied corpus of 1574 documents, updating model once every 1574 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2024-05-30 00:17:00,976 : INFO : PROGRESS: pass 0, at document #1574/1574
2024-05-30 00:17:01,577 : INFO : optimized alpha [0.09927274, 0.10377286, 0.11771142, 0.109483734, 0.094269246, 0.15856536]
2024-05-30 00:17:01,585 : INFO : topic #4 (0.094): 0.027*"스트레스" + 0.021*"때문" + 0.021*"비" + 0.015*"해" + 0.014*"정도" + 0.014*"날씨" + 0.012*"정말" + 0.012*"우울증" + 0.012*"겨울" + 0.011*"사람"
2024-05-30 00:17:01,586 : INFO : topic #0 (0.099): 0.044*"스트레스" + 0.014*"수업" + 0.013*"학생" + 0.012*"때문" + 0.009*"처음" + 0.009*"생활" + 0.008*"영어" + 0.007*"시험" + 0.007*"우울증" + 0.007*"마음"
2024-05-30 00:17:01,587 : INFO : topic #3 (0.109): 0.060*"스트레스" + 0.014*"때문" + 0.014*"정말" + 0.013*"친구" + 0.012*"사람" + 0.010*"수업" + 0.009*"날씨" + 0.009*"생활" + 0.008

In [20]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2024-05-29 23:35:04,959 : INFO : CorpusAccumulator accumulated stats from 1000 documents


Average topic coherence: -2.9197.
[([(0.07069791, '스트레스'),
   (0.022119414, '수업'),
   (0.022052549, '때문'),
   (0.019808007, '생각'),
   (0.015179626, '처음'),
   (0.014645714, '친구'),
   (0.011007964, '공부'),
   (0.010853625, '곳'),
   (0.01073848, '학생'),
   (0.009611201, '정도'),
   (0.0090050865, '한국'),
   (0.008572607, '영어'),
   (0.0074654547, '정말'),
   (0.006894797, '경우'),
   (0.0067192926, '사람'),
   (0.006436092, '말'),
   (0.006040058, '과목'),
   (0.0057948544, '생활'),
   (0.0052802935, '마음'),
   (0.0052265497, '학교')],
  -2.356734566646646),
 ([(0.06714012, '스트레스'),
   (0.019174812, '생활'),
   (0.018194068, '때문'),
   (0.013096796, '교환학생'),
   (0.010941129, '친구'),
   (0.010502788, '생각'),
   (0.00998381, '처음'),
   (0.009697858, '학교'),
   (0.00891774, '영어'),
   (0.007949202, '곳'),
   (0.007576376, '사람'),
   (0.00755063, '경험'),
   (0.0067726523, '수업'),
   (0.006600158, '경우'),
   (0.006546673, '환경'),
   (0.0065243356, '문제'),
   (0.00619517, '개인'),
   (0.0061287093, '기간'),
   (0.005615031, '미국'),
 

### 결과 시각화

In [21]:
!pip install pyLDAvis

import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'display_topics_6개.html')